In [ ]:
from common_imports import *
from db_connection import get_engine
from frequency_utils import (
    get_freq_minutes, get_freq_seconds, get_pandas_freq, check_max_rows,
    round_datetime_to_freq, detect_auto_frequency, resample_dataframe,
)
from progress_bar_widget import ProgressBarWidget
from mappings import get_typeids, validate_unique_ids, group_typeid_mapping
from caching import TTLCache

# --- nieuwe utils -------------------------------------------
from time_utils import (
    DATETIME_FORMAT,
    parse_user_datetime,
    round_datetime_to_freq,      # alleen waar nodig
)

from db_utils import (
    fetch_typeids_for_ean,
    fetch_min_max_period,
    fetch_full_data,
)

from dataset_utils import (
    group_columns_by_typeid,     # alléén in 001_All_Types.ipynb
    build_dataset,
    export_dataset_to_csv,
    export_dataset_to_excel,     # alléén in 002_Data_Export.ipynb
    generate_insights_html,      # alléén in 002_Data_Export.ipynb
)
# -------------------------------------------------------------

show_home_button()

engine = get_engine()
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

progress_widget = ProgressBarWidget()
validate_unique_ids()
MAX_ROWS = 8000

# ─── UI-CONTROLS ──────────────────────────────────────────────────────────────
common_layout = widgets.Layout(width='240px', height='35px')
start_datetime_input = widgets.Text(
    value='01/01/2024 00:00',
    placeholder='dd/mm/yyyy HH:MM',
    description='StartDatum:',
    layout=common_layout
)
end_datetime_input = widgets.Text(
    value='31/12/2024 00:00',
    placeholder='dd/mm/yyyy HH:MM',
    description='EindDatum:',
    layout=common_layout
)
freq_selector = widgets.Dropdown(
    options=[('Automatisch (standaard)', 'auto'),
             ('Elke 5 minuten', '5T'), ('Elke 15 minuten', '15T'),
             ('Per uur', 'H'), ('Dagelijks', 'D'),
             ('Wekelijks', 'W'),
             ('Maandelijks', 'ME'), 
             ('Jaarlijks', 'Y')],
    value='auto',
    description='Frequentie:',
    layout=common_layout
)
aggregate_checkbox = widgets.Checkbox(
    value=True,
    description='Geaggregeerd?',
    layout=widgets.Layout(margin='2px 0 2px 0')
)
status_checkbox = widgets.Checkbox(
    value=False,
    description='Include Status? (en pas kleur op consumption aan)',
    disabled=True,
    layout=widgets.Layout(margin='2px 0 2px 0')
)
excel_format_checkbox = widgets.Checkbox(
    value=False,
    description='Voorwaardelijke opmaak Excel?',
    layout=widgets.Layout(margin='2px 0 2px 0')
)
warning_message = widgets.HTML("")
warning_container = widgets.VBox([])
quick_fix_freq_button = widgets.Button(
    description="Wijzig freq naar 1 uur",
    button_style="warning",
    icon="clock-o",
    layout=common_layout
)
quick_fix_date_button = widgets.Button(
    description="Beperk datumbereik",
    button_style="warning",
    icon="calendar",
    layout=common_layout
)
output_area = widgets.Output(layout={'border': '1px solid black'})
data_table_output = widgets.Output(layout={
    'border': '1px solid #ccc', 'overflow_x': 'auto',
    'overflow_y': 'auto', 'max_height': '400px', 'width': '100%'
})
insights_output = widgets.Output(layout={
    'border': '1px solid #ccc', 'overflow_x': 'auto',
    'overflow_y': 'auto', 'max_height': '400px', 'width': '100%'
})
view_tab = widgets.Tab(children=[data_table_output, insights_output])
view_tab.set_title(0, "Dataset")
view_tab.set_title(1, "Inzichten")
btn_load_filters = widgets.Button(description='Zoeken', button_style='info', icon='filter', layout=common_layout)
btn_build_dataset = widgets.Button(description='Laad Dataset', button_style='success', icon='database', disabled=True, layout=common_layout)
btn_view_dataset = widgets.Button(description="Bekijk Dataset", button_style='primary', icon='eye', disabled=True, layout=common_layout)
btn_view_insights = widgets.Button(description="Bekijk Inzichten", button_style='primary', icon='info', disabled=True, layout=common_layout)
btn_download_csv = widgets.Button(description="Download CSV", button_style='primary', icon='download', disabled=True, layout=common_layout)
btn_download_excel = widgets.Button(description="Download XLS", button_style='primary', icon='file-excel-o', disabled=True, layout=common_layout)
btn_reset_filters = widgets.Button(description='Reset Filters', button_style='warning', icon='refresh', layout=common_layout)
ean_input = widgets.Text(
    description='EAN:',
    placeholder='Vul ID/EAN in',
    value='',
    layout=common_layout
)
options_container = widgets.VBox([aggregate_checkbox, status_checkbox, excel_format_checkbox])
options_accordion = widgets.Accordion(children=[options_container])
options_accordion.set_title(0, "Opties")
options_accordion.selected_index = None
group_checkbox_container = widgets.VBox([])
group_accordion = widgets.Accordion(children=[group_checkbox_container])
group_accordion.set_title(0, "Beschikbare groepen")
group_accordion.selected_index = None
row_top = widgets.HBox(
    [ean_input, btn_load_filters, btn_reset_filters],
    layout=widgets.Layout(gap="5px", flex_flow='row wrap')
)
row_dates = widgets.HBox([start_datetime_input, end_datetime_input, freq_selector],
                         layout=widgets.Layout(gap="10px", flex_flow='row wrap'))
action_buttons_row = widgets.HBox(
    [btn_build_dataset, btn_view_dataset, btn_view_insights, btn_download_csv, btn_download_excel],
    layout=widgets.Layout(justify_content='flex-start', flex_flow='row wrap')
)
toggle_filters_button = widgets.Button(
    description="Verberg filters",
    icon='chevron-up',
    button_style='info',
    layout=widgets.Layout(width='150px', height='35px')
)
filters_container = widgets.VBox([
    widgets.HTML("<h3>Filters</h3>"),
    row_top,
    row_dates,
    warning_container,
    options_accordion,
    group_accordion,
    action_buttons_row,
    progress_widget.widget(),
    output_area,
    view_tab
], layout=widgets.Layout(display='block', border='1px solid #ccc', padding='5px'))
final_ui = widgets.VBox([
    toggle_filters_button,
    filters_container
])
display(final_ui)

current_df = None

# ─── Interacties ──────────────────────────────────────────────────────────────

def toggle_filters_display(b):
    if filters_container.layout.display == 'none':
        filters_container.layout.display = 'block'
        toggle_filters_button.description = "Verberg filters"
        toggle_filters_button.icon = "chevron-up"
    else:
        filters_container.layout.display = 'none'
        toggle_filters_button.description = "Toon filters"
        toggle_filters_button.icon = "chevron-down"
toggle_filters_button.on_click(toggle_filters_display)

def adjust_dates_on_freq_change(change):
    freq_value = change['new'] if isinstance(change, dict) else change
    if freq_value == 'auto':
        return
    start_dt_str = start_datetime_input.value
    end_dt_str = end_datetime_input.value
    start_dt_parsed = parse_user_datetime(start_dt_str)
    end_dt_parsed = parse_user_datetime(end_dt_str)
    if start_dt_parsed is None or end_dt_parsed is None:
        logger.warning("Ongeldige start/einddatum bij aanpassen frequentie.")
        return
    adjusted_start_dt = round_datetime_to_freq(start_dt_parsed, freq_value, is_start=True)
    adjusted_end_dt = round_datetime_to_freq(end_dt_parsed, freq_value, is_start=False)
    if adjusted_start_dt:
        start_datetime_input.value = adjusted_start_dt.strftime(DATETIME_FORMAT)
    if adjusted_end_dt:
        end_datetime_input.value = adjusted_end_dt.strftime(DATETIME_FORMAT)

def validate_data_request(change=None):
    start_dt = parse_user_datetime(start_datetime_input.value)
    end_dt = parse_user_datetime(end_datetime_input.value)
    current_warnings = []
    if not start_dt:
        current_warnings.append("Ongeldige startdatum/tijd (formaat dd/mm/yyyy HH:MM)!")
    if not end_dt:
        current_warnings.append("Ongeldige einddatum/tijd (formaat dd/mm/yyyy HH:MM)!")
    if start_dt and end_dt and end_dt < start_dt:
        current_warnings.append("Einddatum mag niet vóór de startdatum liggen.")
    freq_val = freq_selector.value
    if freq_val.lower() != 'auto' and start_dt and end_dt and end_dt >= start_dt:
        is_ok, expected_rows = check_max_rows(start_dt, end_dt, freq_val, max_rows=MAX_ROWS)
        if not is_ok:
            warning_text = (
                f"U probeert te veel data op te vragen ({int(expected_rows)} rijen bij freq '{freq_val}'). "
                "Verklein het datumbereik of verhoog de resolutie (bijv. naar 'Per uur')."
            )
            current_warnings.append(warning_text)
            warning_container.children = [
                widgets.HTML(f"<span style='color:red; font-weight:bold;'>{'; '.join(current_warnings)}</span>"),
                widgets.HBox([quick_fix_freq_button, quick_fix_date_button], layout=widgets.Layout(justify_content='center'))
            ]
            return
    if current_warnings:
        warning_message.value = f"<span style='color:red; font-weight:bold;'>{'; '.join(current_warnings)}</span>"
        quick_fix_buttons_display = []
        if any("te veel data" in warn_msg for warn_msg in current_warnings):
            quick_fix_buttons_display = [widgets.HBox([quick_fix_freq_button, quick_fix_date_button], layout=widgets.Layout(justify_content='center'))]
        warning_container.children = [warning_message] + quick_fix_buttons_display
    else:
        warning_message.value = ""
        warning_container.children = []

def on_aggregate_change(change):
    status_checkbox.disabled = change['new']
aggregate_checkbox.observe(on_aggregate_change, names='value')
status_checkbox.disabled = aggregate_checkbox.value

def on_freq_change_and_validate(change):
    adjust_dates_on_freq_change(change)
    validate_data_request(change)

start_datetime_input.observe(validate_data_request, names="value")
end_datetime_input.observe(validate_data_request, names="value")
freq_selector.observe(on_freq_change_and_validate, names='value')

def quick_fix_freq_action(b):
    freq_selector.value = 'H'
quick_fix_freq_button.on_click(quick_fix_freq_action)

def quick_fix_date_action(b):
    start_dt = parse_user_datetime(start_datetime_input.value)
    if not start_dt:
        return
    freq_val = freq_selector.value
    if freq_val == 'auto':
        logger.info("Quick fix date: Freq is 'auto', using 'H' for calculation.")
        freq_val = 'H'
    seconds_per_interval = get_freq_seconds(freq_val)
    if seconds_per_interval <= 0:
        seconds_per_interval = 3600
    max_duration = (MAX_ROWS - 1) * seconds_per_interval
    new_end_dt = start_dt + timedelta(seconds=max_duration)
    end_datetime_input.value = new_end_dt.strftime(DATETIME_FORMAT)
    validate_data_request()
quick_fix_date_button.on_click(quick_fix_date_action)

def load_filters_thread(ean_val: str):
    btn_build_dataset.disabled = True
    btn_view_dataset.disabled = True
    btn_view_insights.disabled = True
    btn_download_csv.disabled = True
    btn_download_excel.disabled = True
    with output_area:
        clear_output()
        print("Filters worden geladen...")
    typeids = fetch_typeids_for_ean(ean_val, search_method='transferpoint', engine=engine)
    if not typeids:
        with output_area:
            clear_output()
            print("Geen TypeIDs gevonden voor deze invoer.")
        group_checkbox_container.children = []
        group_accordion.selected_index = None
        return
    relevant_groups = []
    for grp, tlist in group_typeid_mapping.items():
        if set(tlist).intersection(typeids):
            relevant_groups.append(grp)
    if not relevant_groups:
        with output_area:
            clear_output()
            print("Geen relevante groepen gevonden voor de TypeIDs.")
        group_checkbox_container.children = []
        group_accordion.selected_index = None
        return
    checkboxes = []
    for grp in sorted(relevant_groups):
        cb = widgets.Checkbox(value=True, description=grp, indent=False)
        checkboxes.append(cb)
    group_checkbox_container.children = checkboxes
    group_accordion.selected_index = 0
    btn_build_dataset.disabled = False
    with output_area:
        clear_output()
        print(f"Filters geladen. Beschikbare groepen: {', '.join(sorted(relevant_groups))}")

def on_load_filters_clicked(b):
    ean_val = ean_input.value.strip()
    if not ean_val:
        with output_area:
            clear_output()
            print("Vul een EAN of ID in.")
        return
    threading.Thread(target=load_filters_thread, args=(ean_val,)).start()

def on_reset_filters_clicked(b):
    if group_checkbox_container.children:
        for cb in group_checkbox_container.children:
            cb.value = True
    ean_input.value = ''
    start_datetime_input.value = '01/01/2024 00:00'
    end_datetime_input.value = '31/12/2024 00:00'
    freq_selector.value = 'auto'
    aggregate_checkbox.value = True
    status_checkbox.value = False
    excel_format_checkbox.value = False
    status_checkbox.disabled = aggregate_checkbox.value
    with output_area:
        clear_output()
        print("Filters zijn gereset.")
    validate_data_request()

def get_selected_groups() -> list:
    return [cb.description for cb in group_checkbox_container.children if cb.value]

def build_dataset_thread():
    global current_df
    btn_build_dataset.disabled = True
    btn_view_dataset.disabled = True
    btn_view_insights.disabled = True
    btn_download_csv.disabled = True
    btn_download_excel.disabled = True
    progress_widget.show(status="Dataset wordt opgebouwd...")
    with output_area:
        clear_output(wait=True)
    ean_val = ean_input.value.strip()
    if not ean_val:
        with output_area:
            print("Vul een EAN/ID in.")
        progress_widget.update(100, "Fout: EAN/ID ontbreekt", error=True)
        progress_widget.finish()
        btn_build_dataset.disabled = False
        return
    validate_data_request()
    if warning_container.children:
        with output_area:
            display(HTML("<span style='color:red; font-weight:bold;'>Los de validatiefouten op voordat u de dataset bouwt.</span>"))
        progress_widget.update(100, "Validatiefout", error=True)
        progress_widget.finish()
        btn_build_dataset.disabled = False
        return
    start_dt = parse_user_datetime(start_datetime_input.value)
    end_dt = parse_user_datetime(end_datetime_input.value)
    if not start_dt or not end_dt:
        with output_area:
            print("Interne fout: Ongeldige datums ondanks validatie.")
        progress_widget.update(100, "Fout: Interne datumfout", error=True)
        progress_widget.finish()
        btn_build_dataset.disabled = False
        return
    freq_val = freq_selector.value
    agg_val = aggregate_checkbox.value
    status_val = status_checkbox.value
    chosen = get_selected_groups()
    if not chosen:
        with output_area:
            print("Geen groepen geselecteerd.")
        progress_widget.update(100, "Fout: Geen groepen", error=True)
        progress_widget.finish()
        btn_build_dataset.disabled = False
        return
    progress_widget.update(30, "TypeIDs en min/max periode ophalen...")
    df_resampled = build_dataset(
        ean_val,
        chosen,
        start_dt,
        end_dt,
        freq_val,
        agg_val,
        include_status_raw=status_val,
        search_method='transferpoint',
        engine=engine,
    )
    progress_widget.update(80, "Dataset verwerken...")
    if df_resampled is None or df_resampled.empty:
        progress_widget.update(100, "Geen dataset opgehaald.", error=True)
        with output_area:
            print("Geen dataset opgehaald. Controleer filters of logs voor details.")
        progress_widget.finish()
        btn_build_dataset.disabled = False
        return
    current_df = df_resampled.copy()
    progress_widget.update(100, f"Dataset geladen met {len(current_df)} rijen.")
    with output_area:
        clear_output(wait=True)
        print(f"Dataset succesvol geladen met {len(current_df)} rijen.\nU kunt nu de dataset bekijken, inzichten genereren of downloaden.")
    btn_view_dataset.disabled = False
    btn_view_insights.disabled = False
    btn_download_csv.disabled = False
    btn_download_excel.disabled = False
    progress_widget.finish()
    btn_build_dataset.disabled = False

def on_build_dataset_clicked(b):
    with data_table_output: clear_output()
    with insights_output: clear_output()
    threading.Thread(target=build_dataset_thread).start()

def show_dataset_table():
    with data_table_output:
        clear_output(wait=True)
        if current_df is None or current_df.empty:
            print("Nog geen dataset geladen of de dataset is leeg.")
        else:
            limit = 50
            html_table = current_df.head(limit).to_html(classes="dataframe", index=False, escape=True)
            info_message = ""
            if len(current_df) > limit:
                info_message = f"<p><i>Toont de eerste {limit} rijen van {len(current_df)} totale rijen. Download voor de volledige dataset.</i></p>"
            scrollable_html = f"""
            <div style="overflow-x: auto; overflow-y: auto; max-height: 380px; width: 100%;">
                {html_table}
            </div>
            {info_message}
            """
            display(HTML(scrollable_html))
    view_tab.selected_index = 0

def show_insights():
    with insights_output:
        clear_output(wait=True)
        if current_df is None or current_df.empty:
            print("Nog geen dataset geladen of de dataset is leeg om inzichten te genereren.")
        else:
            insights_html = generate_insights_html(current_df)
            display(HTML(f"<div style='font-family: Roboto, sans-serif; font-size:14px;'>{insights_html}</div>"))
    view_tab.selected_index = 1

def on_view_dataset_clicked(b):
    show_dataset_table()

def on_view_insights_clicked(b):
    show_insights()

def on_download_csv_clicked(b):
    if current_df is None or current_df.empty:
        with output_area:
            clear_output(wait=True)
            print("Geen dataset om te downloaden.")
        return

    ean_val = ean_input.value.strip().replace(" ", "_").replace("/", "_")
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename_base = f"dataset_{ean_val}_{ts}.csv"

    downloads_folder = os.path.join(os.path.expanduser("~"), "Downloads")
    if not os.path.isdir(downloads_folder):
        try:
            os.makedirs(downloads_folder)
        except OSError:
            logger.warning(
                f"Kon map {downloads_folder} niet aanmaken. CSV wordt opgeslagen in huidige werkmap."
            )
            downloads_folder = os.getcwd()

    filename = os.path.join(downloads_folder, filename_base)

    if export_dataset_to_csv(current_df, filename):
        with output_area:
            clear_output(wait=True)
            display(
                HTML(
                    f"<p>CSV bestand opgeslagen in uw Downloads map: <code>{filename}</code></p>"
                    f"<p>Als de download niet automatisch start, kunt u het bestand hier vinden.</p>"
                )
            )
    else:
        with output_area:
            clear_output(wait=True)
            print("Fout bij exporteren naar CSV.")


def on_download_excel_clicked(b):
    if current_df is None or current_df.empty:
        with output_area:
            clear_output(wait=True)
            print("Geen dataset om te downloaden.")
        return

    ean_val = ean_input.value.strip().replace(" ", "_").replace("/", "_")
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename_base = f"dataset_{ean_val}_{ts}.xlsx"

    downloads_folder = os.path.join(os.path.expanduser("~"), "Downloads")
    if not os.path.isdir(downloads_folder):
        try:
            os.makedirs(downloads_folder)
        except OSError:
            logger.warning(
                f"Kon map {downloads_folder} niet aanmaken. Excel wordt opgeslagen in huidige werkmap."
            )
            downloads_folder = os.getcwd()

    filename = os.path.join(downloads_folder, filename_base)

    apply_excel_format = excel_format_checkbox.value
    include_status_for_formatting = status_checkbox.value and not aggregate_checkbox.value

    # **HIER DE FIX: keyword-only argumenten gebruiken**
    if export_dataset_to_excel(
        current_df,
        filename,
        excel_format=apply_excel_format,
        include_status=include_status_for_formatting,
    ):
        with output_area:
            clear_output(wait=True)
            display(
                HTML(
                    f"<p>Excel bestand opgeslagen in uw Downloads map: <code>{filename}</code></p>"
                    f"<p>Als de download niet automatisch start, kunt u het bestand hier vinden.</p>"
                )
            )
    else:
        with output_area:
            clear_output(wait=True)
            print("Fout bij exporteren naar Excel.")

btn_load_filters.on_click(on_load_filters_clicked)
btn_reset_filters.on_click(on_reset_filters_clicked)
btn_build_dataset.on_click(on_build_dataset_clicked)
btn_view_dataset.on_click(on_view_dataset_clicked)
btn_view_insights.on_click(on_view_insights_clicked)
btn_download_csv.on_click(on_download_csv_clicked)
btn_download_excel.on_click(on_download_excel_clicked)

# Initial validation call to check default dates
validate_data_request()
adjust_dates_on_freq_change({'new': freq_selector.value})
validate_data_request()


ModuleNotFoundError: No module named 'common_imports'